# All modules fast check 

In [1]:
import main_function as mf


## Set batch name and get data form files
1. 'old' last batch of cells
2. 'Dec' YiHui's new cells from Dec


In [2]:
#mf.batch_name = 'Old'
mf.batch_name = 'Dec'
#mf.batch_name = 'gold-copper'

data = mf.read_all_test()

## Draw overall PCE chart

In [3]:
mf.draw_cell_PCE_overview(data)

## Draw overall PCE ratio chart

In [4]:
mf.draw_cell_PCE_overview_ratio(data)

## Draw all cell performance change ratio

In [5]:
mf.draw_cell_performance(data)

,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.784523,16.709150,0.766592,22.858142,1.775417,16.668357,0.726099,21.487671
1,12.0,1.799366,17.004645,0.703546,21.526800,1.746896,16.837634,0.698576,20.547625
2,24.0,1.810691,17.063457,0.709711,21.927699,1.763129,16.955761,0.718288,21.473353
3,48.0,1.620461,15.888785,0.672164,17.306313,1.580667,15.838527,0.647660,16.214438


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.766466,17.199706,0.728413,22.131145,1.758381,17.184029,0.697808,21.085024
1,12.0,1.666381,17.632720,0.636161,18.692203,1.628576,17.641623,0.663258,19.055871
2,24.0,1.692920,17.425815,0.641825,18.934165,1.657608,17.398545,0.676585,19.512686
3,48.0,1.636802,17.157988,0.595360,16.720214,1.612799,17.102064,0.608408,16.781219


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.798942,17.545679,0.788679,24.893588,1.787580,17.553793,0.759456,23.830829
1,12.0,1.814791,19.146034,0.740484,25.728882,1.793646,19.122152,0.742640,25.471334
2,36.0,1.811039,18.509345,0.749275,25.116566,1.791632,18.461736,0.750129,24.811764
3,96.0,1.609700,23.257971,0.675390,25.285508,1.577441,23.292445,0.672279,24.701200
4,120.0,1.598764,22.180510,0.665296,24.003516,1.579841,22.196827,0.668356,23.437555
5,132.0,1.577694,23.309881,0.668060,24.568475,1.553102,23.249707,0.663021,23.941146
6,144.0,1.531036,22.597945,0.657468,22.747242,1.506146,22.671316,0.651827,22.257490
7,156.0,1.549105,24.634912,0.638577,24.369415,1.520063,24.672889,0.633799,23.770230
8,204.0,1.631911,21.929608,0.671850,24.043617,1.590899,23.011020,0.643638,23.562438
9,252.0,1.610796,22.030145,0.661352,23.468800,1.581303,22.138519,0.638873,22.365488


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.768557,17.931111,0.768563,24.372811,1.758702,17.851587,0.740620,23.252212
1,12.0,1.637508,16.595594,0.661920,17.987959,1.599648,16.502564,0.648443,17.117789
2,36.0,1.637878,16.833704,0.668552,18.433012,1.603937,16.859697,0.632813,17.112469
3,96.0,1.551924,15.807089,0.611954,15.012082,1.473602,15.673326,0.527142,12.174999


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.736775,17.643273,0.752230,23.050132,1.733186,17.629388,0.723039,22.092471
1,12.0,1.648856,21.209006,0.725611,25.375055,1.614510,21.277319,0.728289,25.018506
2,264.0,0.680456,18.958461,0.748722,9.658810,0.000000,19.098052,0.000000,0.000000


...

,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.791719,17.041000,0.783588,23.925047,1.773795,17.008874,0.795465,23.999377
1,12.0,1.819699,16.147766,0.816317,23.986715,1.820261,16.149813,0.813084,23.902138
2,24.0,1.814586,16.544555,0.823186,24.713289,1.803459,16.536552,0.815942,24.333823
3,240.0,1.556255,9.663792,0.419339,6.306569,1.553742,9.899909,0.385714,5.933019
4,288.0,1.410592,7.356804,0.381471,3.958693,1.415291,7.433422,0.364311,3.832714
5,348.0,1.439675,8.223127,0.395787,4.685573,1.415461,8.132458,0.383251,4.411671


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.791719,17.041000,0.783588,23.925047,1.773795,17.008874,0.795465,23.999377
1,12.0,1.819699,16.147766,0.816317,23.986715,1.820261,16.149813,0.813084,23.902138
2,24.0,1.814586,16.544555,0.823186,24.713289,1.803459,16.536552,0.815942,24.333823
3,240.0,1.556255,9.663792,0.419339,6.306569,1.553742,9.899909,0.385714,5.933019
4,288.0,1.410592,7.356804,0.381471,3.958693,1.415291,7.433422,0.364311,3.832714


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.820121,18.055191,0.815466,26.798367,1.803335,18.063653,0.816678,26.603125
1,12.0,1.777663,17.659802,0.806019,25.303499,1.778983,17.651767,0.807473,25.356437
2,24.0,1.766478,17.953500,0.808240,25.632907,1.767439,17.968915,0.806317,25.607789
3,240.0,1.767483,18.165353,0.810097,26.009741,1.761788,18.142811,0.814981,26.049874
4,288.0,1.411253,14.777257,0.379556,7.915432,1.376144,14.566319,0.361800,7.252410
5,348.0,1.208892,12.578954,0.408771,6.216007,1.252875,12.635788,0.367324,5.815130


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.820121,18.055191,0.815466,26.798367,1.803335,18.063653,0.816678,26.603125
1,12.0,1.777663,17.659802,0.806019,25.303499,1.778983,17.651767,0.807473,25.356437
2,24.0,1.766478,17.953500,0.808240,25.632907,1.767439,17.968915,0.806317,25.607789
3,240.0,1.767483,18.165353,0.810097,26.009741,1.761788,18.142811,0.814981,26.049874
4,288.0,1.411253,14.777257,0.379556,7.915432,1.376144,14.566319,0.361800,7.252410


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.734389,17.139273,0.632049,18.788400,1.731428,16.920977,0.621731,18.215150
1,12.0,1.692983,16.722894,0.654386,18.526708,1.688073,16.701698,0.630847,17.785891
2,36.0,1.677338,16.619049,0.622208,17.344528,1.676280,16.604802,0.596462,16.602095
3,96.0,1.679058,16.996748,0.587786,16.774540,1.675122,17.046033,0.561235,16.025620
4,108.0,1.683115,16.541922,0.606170,16.876971,1.677031,16.426996,0.580048,15.979508
5,120.0,1.683473,16.763295,0.581258,16.403424,1.676419,16.841652,0.542669,15.321544
6,144.0,1.672677,17.040290,0.544510,15.520106,1.654301,16.900963,0.500559,13.995262
7,264.0,1.719085,15.752624,0.552615,14.964869,1.713017,15.591105,0.519267,13.868495
8,276.0,1.546437,15.885149,0.436158,10.714385,1.525187,16.076512,0.402756,9.875442
